# Linear Solver Performance: Constant Coefficient Poisson, MPI-parallel

### Part 1, Benchmark Setup and Execution

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

In [ ]:
#r "BoSSSpad.dll"
//#r "../../../../src/L4-application/BoSSSpad/bin/Debug/net6.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Platform.Utils.Geom;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.GridGen;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
using BoSSS.Application.SipPoisson;

In [ ]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfPar") ?? "LinslvPerfPar"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

In [ ]:
ExecutionQueues

In [ ]:
GetDefaultQueue()

## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
}

## Init grids and save to database

In [ ]:
wmg.Grids

Create meshes in various resolutions:
- domain $\Omega = (0,10) \times (-1,1) \times (-1,1)$; 
- a Dirichlet boundary is set for $x = 0$, everywhere else Neumann boundaries are assumed; 
  these are typically more challenging for a linear solver than Dirichlet boundaries.
- in order to make the problem a bit challenging, a non-equidistant spacing for $y$ and $z$ coordinate are used
- the domain shape is reminiscent of a typical grid for the simulation of channel flows, with exception of the boundary conditions

Since some of the grids are quite large, it is unhandy to create them within this worksheet. 
Instead, the `GridGen` app is used to create the grids in parallel on the HPC System.

In [ ]:
int[] Resolutions_3D = new int[] { 4, 8, 16, 24, 32, 48, 64, 128, 256 };
string[] GridNameS = new string[Resolutions_3D.Length];
var ggcS = new (GridGenControl C, int MPIsize)[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] _xNodes = GenericBlas.Linspace(0, 10, Res*5 + 1);
    double[] _yNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    double[] _zNodes = Utils.SinLinSpacing(-1, +1, 0.6, Res + 1);
    int J = (_xNodes.Length - 1)*(_yNodes.Length - 1)*(_zNodes.Length - 1);
    
    string GridName = wmg.CurrentProject + "-SIP_Poisson_J" + J;
    GridNameS[cnt] = GridName;
    
    if(wmg.Grids.Where(grd => grd.Name.Contains(GridName)).Count() <= 0) {
        int NoOfProcs = (int) Math.Min(182, Math.Max(1, Math.Ceiling(J/200000.0)));
        Console.WriteLine("Must create: " + GridName + " with " + NoOfProcs + " processors.");
        
        var C = new GridGenControl();
        ggcS[cnt] = (C, NoOfProcs);
        C.SetDatabase(wmg.DefaultDatabase);
        
        C.GridName = GridName;

        C.GridBlocks = new GridGenControl.MeshBlock[] {
            new GridGenControl.Cartesian3D() {
                xNodes = _xNodes,
                yNodes = _yNodes,
                zNodes = _zNodes
            }
        };

        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -1e-6, -2, -2 }, new double[] { +1e-6, +2, +2 }), 
            BoundaryType.Dirichlet.ToString()));
        C.BoundaryRegions.Add((
            null, 
            BoundaryType.Neumann.ToString()));
        
        C.SessionName = "GridCreation-" + GridName;
    } else {
        Console.WriteLine("Found grid: " + GridName);
    }
}

In [ ]:
foreach(var tt in ggcS) {
    if(tt.C != null) {
    Console.WriteLine(" Submitting: " + tt.C.SessionName); 
    var j = tt.C.CreateJob();
    j.RetryCount = 2;
    j.NumberOfMPIProcs = tt.MPIsize;
    j.Activate();
    }
}

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish
wmg.AllJobs

In [ ]:
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    int J = Res*5*Res*Res;
    
    Console.WriteLine("Searching for grid with " + J + " cells");
    grids[cnt] = wmg.Grids.FirstOrDefault(grd => grd.Name.Contains(GridNameS[cnt])); // grid must be present now
    
    if(grids[cnt] != null) {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");

        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    } else {
        Console.Error.WriteLine("missing: J = " + J);
    }
}

In [ ]:
grids

In [ ]:
NUnit.Framework.Assert.IsTrue(grids.All(g => g != null), "Unable to find/create some grid.");

## Setup Control Object for a Solver Run

### Values/Formulas for the Values of RHS and Boundary Conditions

In [ ]:
string nl = System.Environment.NewLine;
string math_expressions_code = 
"static class InitialValues { " +
"    public static double RHS(double[] X) { " + nl +
"        return -Math.Sin(X[0]); " + nl +
"    } " + nl +
"     " + nl +
"    public static double DirichletBC(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"     " + nl +
"    public static double NeumannBC(double[] X) { " + nl +
"       if(Math.Abs(X[1] - 1.0) < 1.0e-8 || Math.Abs(X[1] + 1.0) < 1.0e-8) " + nl +
"           return 0; " + nl +
"       if(X.Length > 2 && (Math.Abs(X[2] - 1.0) < 1.0e-8 || Math.Abs(X[2] + 1.0) < 1.0e-8)) " + nl +
"           return 0; " + nl +
"     " + nl +
"       return Math.Cos(10.0); " + nl +
"   } " + nl +
"} " + nl;

In [ ]:
var RHS_value = new Formula("InitialValues.RHS", false, math_expressions_code);
var NeumannBC_value = new Formula("InitialValues.NeumannBC", false, math_expressions_code);
var DirichletBC_value = new ConstantValue(0.0);

Brief test of the boundary values:

In [ ]:
RHS_value.Evaluate(new Vector(0.2, 0.1), 0)

In [ ]:
NeumannBC_value.Evaluate(new Vector(0.2, 0.1), 0)

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
int[] PolyDegS = new int[] { 2, 3, 5 }; // reduced options for dev.

Number of processors:

In [ ]:
int[] MPIsizes = new int[] { 1, 2, 4, 8, 16, 32, 64, 128 };

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
var controls = new List<(SipControl ctrl, int NoOfProcs)>();
LinearSolverCode solver_name = LinearSolverCode.exp_Kcycle_schwarz;
foreach(int k in PolyDegS) {   
foreach(IGridInfo grd in grids) {
foreach(int MPIsize in MPIsizes) {
    if(grd == null)
        continue;

    int Np = Utils.Np(k);
    int J  = grd.NumberOfCells;
    if(J / MPIsize < 16) {
        // less than 16 cells per processor - to low for a multigrid.
        continue; 
    }
    if(J*Np/MPIsize > 500000) {
        // not interested in doing more then 500'000 DOFs per processor
        continue;
    } 
    var ctrl = new SipControl();
    controls.Add((ctrl, MPIsize));
       
    
    string caseName = string.Format("Poisson_J{0}_k{1}_Sz{2}", J, k, MPIsize);
    Console.WriteLine("setting up: " + caseName);
    ctrl.SessionName = "SIP_" + caseName;

    ctrl.SetGrid(grd);
    ctrl.savetodb = true;
    ctrl.SetDGdegree(k);

    ctrl.LinearSolver           = solver_name.GetConfig();
    var isc = ctrl.LinearSolver as IterativeSolverConfig;
    if(isc != null) {
        //Console.WriteLine(isc.ConvergenceCriterion);
        //ctrl.LinearSolver.TargetBlockSize      = Math.Min(J*Np-1,10000);
        isc.ConvergenceCriterion = 1e-8;
    }
        
        
    ctrl.InitialValues.Add("RHS", RHS_value);
    ctrl.AddBoundaryValue(BoundaryType.Dirichlet.ToString(), "T", DirichletBC_value);
    ctrl.AddBoundaryValue(BoundaryType.Neumann.ToString(), "T", NeumannBC_value);
    
    ctrl.TracingNamespaces = "*";
}
}
}

In [ ]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\ipPoisson\bin\Release\net5.0\PoissonBench";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, ctrl.SessionName + ".obj"));
//}

In [ ]:
controls.Count()

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
int i = 1;
foreach((var ctrl, int MPIsize) in controls) {
    Console.WriteLine($"   Submitting {i}/{controls.Count()}: " + ctrl.SessionName);
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.NumberOfMPIProcs = MPIsize;
    j.Activate(myBatch);
    i++;
    //ctrl.RunBatch(); // run in the default queue for this project
}

In [ ]:
//foreach(var j in wmg.AllJobs.Values)
//    j.DeleteOldDeploymentsAndSessions();

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
int succJob = wmg.AllJobs.Values.Where(job => !job.Name.Contains("GridCreation") && job.Status == JobStatus.FinishedSuccessful).Count();
int succSess = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") && (Si.SuccessfulTermination == true)).Count();
(succJob, succSess)

In [ ]:
int failJob = wmg.AllJobs.Values.Where(job => !job.Name.Contains("GridCreation") && job.Status != JobStatus.FinishedSuccessful).Count();
int failSess = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") && (Si.SuccessfulTermination == false)).Count();
(failJob, failSess)

In [ ]:
wmg.AllJobs.Values.First().Name

In [ ]:
NUnit.Framework.Assert.AreEqual(succJob + failJob, controls.Count);
//NUnit.Framework.Assert.AreEqual(succJob, succSess);
//NUnit.Framework.Assert.AreEqual(failJob, failSess);

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("SIP_Poisson_J"))

In [ ]:
//controls.Select(tuple => tuple.ctrl)
//        .Select(ctrl => (ctrl,ctrl.GetJob()))
//        .Select(ttt => (ttt.Item1.SessionName == ttt.Item2.Name, ttt.Item1.SessionName, ttt.Item2.Name))

In [ ]:
var NoSuccess = wmg.AllJobs.Values
                        .Where(job => job.Status != JobStatus.FinishedSuccessful)
                        .ToArray();
NoSuccess

In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));

In [ ]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("SIP_Poisson") && !Si.Name.Contains("GridCreation") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions

In [ ]:
//foreach(var s in FailedSessions)
//    s.Delete(true);

In [ ]:
//wmg.AllJobs["SIP_Poisson_J2560_k5_pMultigrid"].AllDeployments

In [ ]:
//NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [ ]:
//foreach(Job jF in NoSuccess)
//   jF.DeleteOldDeploymentsAndSessions();

In [ ]:
//NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not derminate successfully.");